In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from tabulate import tabulate
from collections import Counter

In [2]:
df = pd.read_csv("customer_complaints_1.csv")  # Replace with your actual file path

dataset = df["text"].dropna().astype(str).tolist()

In [3]:
def convert_to_lowercase(text):
    return text.lower()
dataset = [convert_to_lowercase(text) for text in dataset]
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: i used to love comcast. until all these constant updates. my internet and cable crash a lot at night, and sometimes during the day, some channels don't even work and on demand sometimes don't play either. i wish they will do something about it. because just a few mins ago, the internet have crashed for about 20 mins for no reason. i'm tired of it and thinking about switching to wow or something. please do not get xfinity.
2: i'm so over comcast! the worst internet provider. i'm taking online classes and multiple times was late with my assignments because of the power interruptions in my area that lead to poor quality internet service. definitely switching to verizon. i'd rather pay $10 extra then dealing w/ comcast and non stopping internet problems.
3: if i could give them a negative star or no stars on this review i would. i have never worked with any industry with as bad of customer service as comcast. it is not a matter of money because i make well enough above and beyond to aff

In [4]:
import re
import string

def remove_punctuation(text):
    if not isinstance(text, str):
        return text
    text = re.sub(r'[\n\r]+', ' ', text)
    punctuation_to_remove = string.punctuation.replace("'", "")
    text = re.sub(r"[{}]".format(re.escape(punctuation_to_remove)), " ", text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

dataset = [remove_punctuation(text) for text in dataset]

for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: i used to love comcast until all these constant updates my internet and cable crash a lot at night and sometimes during the day some channels don't even work and on demand sometimes don't play either i wish they will do something about it because just a few mins ago the internet have crashed for about 20 mins for no reason i'm tired of it and thinking about switching to wow or something please do not get xfinity
2: i'm so over comcast the worst internet provider i'm taking online classes and multiple times was late with my assignments because of the power interruptions in my area that lead to poor quality internet service definitely switching to verizon i'd rather pay 10 extra then dealing w comcast and non stopping internet problems
3: if i could give them a negative star or no stars on this review i would i have never worked with any industry with as bad of customer service as comcast it is not a matter of money because i make well enough above and beyond to afford their services 

In [5]:
import re

def remove_nums_symbols(text):
    if not isinstance(text, str):
        return text
    # Remove numbers
    text = re.sub(r'\d+', ' ', text)
    # Remove all punctuation *except* apostrophes
    text = re.sub(r"[^\w\s']", ' ', text)
    # Remove any lingering underscores (from \w)
    text = re.sub(r'_', ' ', text)
    # Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

dataset = [remove_nums_symbols(text) for text in dataset]

In [6]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    if isinstance(text, str):
        return ' '.join(word for word in text.split() if word.lower() not in stop_words)
    return text
stop_words = set(stopwords.words('english'))
# Apply all transformations
dataset = [remove_stopwords(text) for text in dataset]

# Print the cleaned dataset
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: used love comcast constant updates internet cable crash lot night sometimes day channels even work demand sometimes play either wish something mins ago internet crashed mins reason tired thinking switching wow something please get xfinity
2: comcast worst internet provider taking online classes multiple times late assignments power interruptions area lead poor quality internet service definitely switching verizon rather pay extra dealing w comcast non stopping internet problems
3: could give negative star stars review would never worked industry bad customer service comcast matter money make well enough beyond afford services legitimate ripoff think biggest scam since mortgage industry's major meltdown hope move somewhere comcast exist disregard want help right thing honestly astounding call issues billing connection service adding removing service errors matter transferred minimum times everyone says thing passes issues next person one resolves problem offer promotional packages sm

In [7]:
def remove_suffixes(text):
    if isinstance(text, str):
        words = text.split()
        stripped_words = []
        for word in words:
            # Basic handling for common suffixes
            if word.endswith("ing") and len(word) > 4:
                stripped_words.append(word[:-3])
            elif word.endswith("es") and len(word) > 3:
                stripped_words.append(word[:-2])
            else:
                stripped_words.append(word)
        return ' '.join(stripped_words)
    return text

# Apply the new step AFTER the previous processing
dataset = [remove_suffixes(text) for text in dataset]

# Print the final output
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")

1: used love comcast constant updat internet cable crash lot night sometim day channels even work demand sometim play either wish someth mins ago internet crashed mins reason tired think switch wow someth please get xfinity
2: comcast worst internet provider tak online class multiple tim late assignments power interruptions area lead poor quality internet service definitely switch verizon rather pay extra deal w comcast non stopp internet problems
3: could give negative star stars review would never worked industry bad customer service comcast matter money make well enough beyond afford servic legitimate ripoff think biggest scam since mortgage industry's major meltdown hope move somewhere comcast exist disregard want help right th honestly astound call issu bill connection service add remov service errors matter transferred minimum tim everyone says th pass issu next person one resolv problem offer promotional packag small timefram never access upgrade without wish change bill months 

In [8]:
!pip install autocorrect

from autocorrect import Speller

spell = Speller(lang='en')

def autocorrect_text(text):
    if isinstance(text, str):
        return ' '.join(spell(word) for word in text.split())
    return text

# Apply autocorrect
dataset = [autocorrect_text(text) for text in dataset]

# Print results
print("\nAutocorrected dataset:")
for i, text in enumerate(dataset, start=1):
    print(f"{i}: {text}")


Autocorrected dataset:
1: used love comcast constant update internet cable crash lot night sometime day channels even work demand sometime play either wish some mins ago internet crashed mins reason tired think switch wow some please get finite
2: comcast worst internet provider tak online class multiple tim late assignments power interruptions area lead poor quality internet service definitely switch verizon rather pay extra deal w comcast non stop internet problems
3: could give negative star stars review would never worked industry bad customer service comcast matter money make well enough beyond afford service legitimate runoff think biggest scam since mortgage industry's major meltdown hope move somewhere comcast exist disregard want help right th honestly around call issue bill connection service add remove service errors matter transferred minimum tim everyone says th pass issue next person one resolve problem offer promotional package small timeframe never access upgrade witho

In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset)

In [10]:
k = 2 # Define the number of clusters
km = KMeans(n_clusters=k)
km.fit(X)
# Predict the clusters for each document
y_pred = km.predict(X)
# Display the document and its predicted cluster in a table
table_data = [["Document", "Predicted Cluster"]]
table_data.extend([[doc, cluster] for doc, cluster in zip(dataset, y_pred)])
print(tabulate(table_data, headers="firstrow"))

Document                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [11]:
# Print top terms per cluster
print("\nTop terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(k):
 print("Cluster %d:" % i)
 for ind in order_centroids[i, :10]:
     print(' %s' % terms[ind])
 print()


Top terms per cluster:
Cluster 0:
 rude
 contract
 finite
 local
 representative
 rep
 joke
 service
 fe
 bill

Cluster 1:
 service
 internet
 comcast
 maps
 customer
 speed
 would
 day
 call
 get



In [12]:
# Calculate purity
total_samples = len(y_pred)
cluster_label_counts = [Counter(y_pred)]
purity = sum(max(cluster.values()) for cluster in cluster_label_counts) / total_samples
print("Purity:", purity)

Purity: 0.8421052631578947
